In [8]:
from instancePDSVRP import instancePDSVRP
from solver import PDSVRPModel
import copy
import time
import random
import numpy as np
import heuristic as h
import utilities as u

In [9]:
def SISSRs_with_initial_solution_return(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1

    return s_0, s_best

osservare: c depot migliore, c customer migliore, raggiungi ottimi ogni tanto

In [10]:
def test_solver_and_heuristic(solver, heuristic, instances,  sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    results = []
            
    for instance_name in instances:
        print("Solver for instance " + instance_name)
        instance = instancePDSVRP("instances/small_instances/" + instance_name + ".txt") # Assumi che le istanze abbiano una rappresentazione stringa unica
        
        s = solver(instance) 
        s.build_model()
        s.solve()
        cost_solver = s.model.objVal
        elapsed_time_solver = s.model.Runtime

        print("Heuristic")   
        # Esegui la variante
        start_time = time.time()
        init_sol, sol = heuristic(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
        end_time = time.time()
        cost_init_heu = u.cost(instance, init_sol)
        cost_heu =u.cost(instance, sol)
        elapsed_time_heuristic = end_time - start_time

        print("Cost found with heuristic: ", cost_heu)
       
        # Calcola la differenza percentuale dei costi
        cost_difference = ((cost_heu - cost_solver) / cost_solver) * 100
        cost_init_difference = ((cost_init_heu - cost_solver) / cost_solver) * 100
                

        N_t = [i for i in range(1,instance.N) if (instance.w[i] <=  instance.Q_d and ((instance.t_d[i] * 2) <= instance.d_end))]
        drone_eligible_customers = len(N_t)

        customers_served_by_drones = 0
        trucks_used = 0
         
        if cost_heu < cost_solver:
            for drone_tour in sol[0][1]:
                customers_served_by_drones += len(drone_tour)
            for truck_tour in sol[0][0]:
                if len(truck_tour) > 0:
                    trucks_used += 1

        else:
            trucks_used = sum(1 for i in range(1, s.N) if s.x[0, i].x > 0)
            customers_served_by_drones = sum(1 for i in range(1, s.N) for k in range(s.D) if s.y[i, k].x > 0)

        # Memorizza i risultati
        results.append({
            "instance": instance_name,
            "cost_solver": cost_solver,
            "cost_heuristic": cost_heu,
            "cost_difference_percentage": cost_difference,
            "cost_heuristic_initial": cost_init_heu,  
            "cost_difference_initial_percentage": cost_init_difference,             
            "time_solver": elapsed_time_solver,
            "time_heuristic": elapsed_time_heuristic,
            "drone_eligible_customers": drone_eligible_customers,
            "customers_served_by_drones": customers_served_by_drones,
            "trucks_used" : trucks_used,
            "iter_imp": iter_imp,
            "iter_max": iter_max
        })
    
    return results

In [11]:
# Definisci i metodi e le varianti
model = PDSVRPModel
heuristic = SISSRs_with_initial_solution_return

instances = []
for customers_position in ["c", "r", "rc"]:
    for i in ["0", "1", "2"]:
        for depot_position in ["c", "e", "r"]:
            instances.append("20-" + customers_position + "-" + i + "-" + depot_position)

#instances = ["10-c-1-c"]

# Esegui i test e ottieni i risultati
results = test_solver_and_heuristic(model, heuristic, instances, 0.3, 3, 3, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 100, 1000, 2, 2, 9)

# Salva i risultati in un file CSV
u.save_results_to_csv(results, "tests/results/small_instances_results.csv")

Solver for instance 10-c-1-c
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-21
Set parameter Threads to value 8
Set parameter MIPFocus to value 1
Set parameter Presolve to value 2
Set parameter Cuts to value 3
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 8 threads

Optimize a model with 146 rows, 270 columns and 1004 nonzeros
Model fingerprint: 0xfe1b8803
Variable types: 120 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+03]
  Objective range  [1e-01, 4e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 15.1048282
Presolve removed 55 rows and 145 columns
Presolve time: 0.01s
Presolved: 91 rows, 125 columns, 582 nonzeros
Varia